In [1]:
import math
import time

import numpy as np
import pandas as pd

import src.py.config as config
import src.py.dataset as dataset

In [2]:
df = pd.read_csv('data/poisson_sim.csv')
dummy_prog = pd.get_dummies(df.prog)
df['intercept'] = 1
df['general'] = dummy_prog[1]
df['academic'] = dummy_prog[2]
df['vocational'] = dummy_prog[3]

features = ['intercept', 'math', 'vocational', 'academic']
X = df[features].values
Y = df[['num_awards']].values.flatten().astype('float64')

In [3]:
def fitCD(X, Y, b=None, precision=1e-10):
    n, p = X.shape
    std = X.std(axis=0)
    std[std == 0] = 1
    X /= std
    b = np.zeros(p) if b is None else b * std
    sum_update = 1
    nb_iteration = 0
    r = Y - X @ b 
    divisor = n * (1 + np.square(X).mean(axis=0)) # 2np + 3p
    Xj = [row for row in X.T]
    while math.sqrt(sum_update / p) > precision:
        nb_iteration += 1
        sum_update = 0
        for j in range(p):
            update = float(np.dot(r, Xj[j]) / divisor[j]) #  2n + 1
            sum_update += update**2
            b[j] += update
            r -= update * Xj[j] # 2n
    print("number or iterations", nb_iteration, math.sqrt(sum_update / p))
    return b / std

In [4]:
def irls_poisson(X, Y, solver):
    n, p = X.shape
    b = np.zeros(p) # p
    yp = np.zeros(n) # n
    mu = np.ones(n) # n
    previous_ll = n
    current_ll = n
    while previous_ll >= current_ll:
        previous_ll = current_ll
        W = np.sqrt(mu) # n
        Z = yp + Y / mu - 1 # 3n
        b = solver(W[:, np.newaxis] * X, W * Z, b) # ?
        yp = X @ b # 2np
        mu = np.exp(yp) # n
        current_ll = -(Y.T @ yp - sum(mu)) # 2n
    return b

In [5]:
irls_poisson(X, Y, fitCD)

number or iterations 1576 9.933620773673003e-11
number or iterations 939 9.839206338178205e-11
number or iterations 1063 9.877450642927196e-11
number or iterations 965 9.87371509075679e-11
number or iterations 624 9.951406951952467e-11
number or iterations 98 9.906425074093217e-11
number or iterations 1 9.584553914760944e-11


array([-5.24712438,  0.0701524 ,  0.36980922,  1.08385914])

In [6]:
cfg = config.Config('./config/model_nbsin_ddea.cfg')
label = cfg.label
path = cfg.get_result_path()
ds = dataset.Dataset(cfg.get_dataset_filename())
ds.load()

In [7]:
includes = set(ds.features) - set(cfg.excludes)
includes = [ds.get_feature_index(i) for i in includes]
includes.sort()

In [8]:
n = ds.size
p = len(ds.features)
n, p

(4459542, 156)

In [9]:
X = dataset.load_data('./dataset/mrh_ddea/features.dat', 'uint8', (n, p))
Xt = dataset.load_data('./dataset/test/mt.dat', 'uint8', (p, n))
exposure = dataset.load_data('./dataset/mrh_ddea/column_anpol.dat', 'float32', (n,))
Y = dataset.load_data('./dataset/mrh_ddea/column_nbsinDDE.dat', 'float32', (n,))

In [10]:
m = 0
for i in includes:
    f= ds.features[i]
    m += len(ds.modalities[f])
m

611

In [11]:
BigX = np.zeros(n * m, dtype='float32').reshape(n, m)
#BigXt = np.zeros(n * m, dtype='float32').reshape(m, n)

In [14]:
k = 0
for i in includes:
    f = ds.features[i]
    for j in range(len(ds.modalities[f])):
        BigX[:, k] = (X[:, i] == j)
        k += 1
        print(k)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [16]:
BigXMean = BigX.mean(axis=0)
BigXMean

array([6.34298742e-01, 5.85777638e-03, 9.15398970e-02, 4.57948372e-02,
       2.16970935e-01, 3.21714650e-03, 2.28229724e-03, 3.83447441e-05,
       6.65609598e-01, 8.97397101e-02, 1.77025795e-01, 2.59194784e-02,
       1.74013376e-02, 2.06855331e-02, 3.61853302e-03, 7.50659347e-01,
       2.40760148e-01, 8.58047791e-03, 2.50936091e-01, 1.64581925e-01,
       1.60894096e-01, 9.89541113e-02, 3.14240783e-01, 1.03929956e-02,
       9.47022140e-01, 5.29778637e-02, 4.30277139e-01, 5.44008136e-01,
       2.57147476e-02, 9.93771136e-01, 6.22889074e-03, 1.75586864e-01,
       4.10867304e-01, 2.41729528e-01, 8.16675776e-04, 6.94266334e-02,
       3.39972116e-02, 8.28179251e-03, 6.28921064e-03, 3.89995216e-03,
       2.64937524e-03, 1.86454121e-03, 4.60047228e-03, 2.18011178e-02,
       1.33231170e-02, 1.36852614e-03, 1.02701131e-03, 5.46244439e-04,
       3.65956861e-04, 2.28947276e-04, 1.73111941e-04, 1.69075662e-04,
       1.12343376e-04, 8.54347818e-05, 2.36347143e-04, 3.69993140e-05,
      

In [17]:
BigXStd = BigX.std(axis=0)

In [18]:
BigX = (BigX - BigXMean) / BigXStd

In [20]:
Y.T @ BigX

array([-3.37253047e+04, -6.89144238e+03, -2.44049824e+04,  4.57286094e+04,
        3.20108906e+04, -6.80745068e+03,  1.85559102e+04,  4.58734570e+03,
       -1.06535559e+03,  3.75428398e+04, -3.01213555e+04,  2.29028438e+04,
       -4.98587109e+03, -1.17652559e+04,  3.28296021e+03,  1.26757607e+04,
       -1.36500762e+04,  3.81962598e+03,  6.36388232e+03, -1.74356602e+04,
       -2.11032676e+04, -1.23714990e+04,  3.54001484e+04, -1.76300137e+04,
       -9.85416016e+03,  9.85416016e+03,  6.40947461e+03, -4.94726758e+03,
       -4.48344189e+03, -3.04358228e+03,  3.04357959e+03,  1.23222578e+04,
       -1.63395791e+04, -4.06398281e+04,  9.76675977e+03,  1.79366309e+04,
        1.92837246e+04,  1.54999902e+04,  1.70774648e+04,  1.71819082e+04,
        1.31328086e+04,  1.38717217e+04,  1.56986348e+04,  2.01330684e+04,
        1.77134902e+04,  1.14824609e+04,  9.01970117e+03,  9.29854395e+03,
        7.08336621e+03,  5.33844385e+03,  5.19099854e+03,  4.61647900e+03,
        5.16487891e+03,  

In [21]:
def lasso_path(X, Y):
    n, p = X.shape
    b = np.zeros(p) # p
    yp = np.zeros(n) # n
    mu = np.ones(n) # n
    W = np.sqrt(mu) # n
    Z = mu * (yp + Y / mu - 1) # 3n
    #XX = W[np.newaxis, :] * Xt
    print('go')
    r = Z.T @ X / n # 2np
    return r
    #b = solver(, W * Z, b) # ?
    yp = X @ b # 2np
    mu = np.exp(yp) # n

In [22]:
c = lasso_path(BigX, Y)

go


KeyboardInterrupt: 

In [125]:
ranked_features = []
k = 0
for i in includes:
    f = ds.features[i]
    t = 0
    for j in range(len(ds.modalities[f])):
        t += c[k]**2
        #print(k, f, ds.modalities[f][j], c[k])
        k += 1
    ranked_features.append((f, math.sqrt(t)))
print(sorted(ranked_features, key=lambda x: x[1]))

[('Random', 0.0003014698482374612), ('HAB_surf_dep', 0.0007424769852361248), ('HAB_inser', 0.0009637336631142631), ('HAB_top_jardin', 0.0013249637403838844), ('CLI_sex', 0.002075820712009406), ('annee', 0.0022893039637658023), ('POL_vol', 0.002376035603661245), ('HAB_cdresid', 0.0031264439820036017), ('HAB_inhab', 0.0031264439820036017), ('HAB_habit', 0.0035439587911467903), ('HAB_anclg', 0.0041716282404074025), ('POL_fract', 0.004264182156684405), ('CLI_nb_enfant', 0.005135271550737963), ('catnat_quant_pp_10', 0.005155300480519095), ('POL_distrib_code', 0.006153697803508539), ('POL_fr', 0.006741750160907875), ('clim_quant_pp_20', 0.006780680256284056), ('CLI_sit_fam2', 0.007265297282027866), ('bdg_quant_freq_20', 0.007312167931262003), ('CLI_cspf', 0.0074129350135859915), ('fire_quant_freq_20', 0.007864914596279892), ('Zone_inc_freq_iris', 0.007899255074394549), ('CLI_age', 0.008236153287527935), ('CLI_tr_age', 0.008305109710257436), ('POL_cdregion', 0.011226316352837532), ('Zone_inc_

In [114]:
del BigX